# Conjunctions (TOLEOS)

In [ ]:
SERVER_URL = "https://vires.services/ows"

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib,cartopy

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
from viresclient import SwarmRequest

## Product information

The [`MM_OPER_CON_EPH_2_`](https://swarmhandbook.earth.esa.int/catalogue/MM_CON_EPH_2_) product contains conjunction information between Swarm (A, B, C), CHAMP, GOCE, GRACE (1, 2), and GRACE-FO (1, 2).

The product is implemented in VirES as two collections, each available as a single flat time series.

`MM_OPER_CON_EPH_2_:crossover` contains the list of times where satellite ground-tracks overlap within a ~7 hour window.

`MM_OPER_CON_EPH_2_:plane_alignment` contains much rarer events, where the planes of different spacecraft are aligned

In [ ]:
request = SwarmRequest(SERVER_URL)
for collection in ("MM_OPER_CON_EPH_2_:crossover", "MM_OPER_CON_EPH_2_:plane_alignment"):
    print(f"{collection}:\n{request.available_measurements(collection)}\n")

## Fetching data

### Crossovers

Let's fetch all the available conjunctions for a given day.

Note that the `start_time` and `end_time` specified are used for a full interval query over both `time_1` and `time_2` given in the outputs.

In [ ]:
request = SwarmRequest(SERVER_URL)
request.set_collection("MM_OPER_CON_EPH_2_:crossover")
request.set_products(
    request.available_measurements("MM_OPER_CON_EPH_2_:crossover")
)
data = request.get_between(
    dt.datetime(2020, 1, 1),
    dt.datetime(2020, 1, 2),
)
df = data.as_dataframe()
df

Pairs of conjunctioning spacecraft are defined with short designations in the `satellite_1` and `satellite_2` variables:

In [ ]:
df["satellite_1"].unique()

In [ ]:
df["satellite_2"].unique()

Each conjunction has a start and end time defined with the `time_1` and `time_2` variables:

In [ ]:
df.iloc[0:5][["time_2", "satellite_1", "satellite_2"]]

We can select all the conjunctions containing a given satellite:

In [ ]:
df_SWA = df.where((df["satellite_1"] == "SWA")|(df["satellite_2"] == "SWA")).dropna()
df_SWA

### Plane alignments

In [ ]:
request = SwarmRequest(SERVER_URL)
request.set_collection("MM_OPER_CON_EPH_2_:plane_alignment")
request.set_products(
    request.available_measurements("MM_OPER_CON_EPH_2_:plane_alignment")
)
data = request.get_between(
    dt.datetime(2000, 1, 1),
    dt.datetime(2022, 1, 1),
)
df = data.as_dataframe()
df

In [ ]:
def alignments(df, sat="SWA"):
    return df.where((df["satellite_1"] == sat)|(df["satellite_2"] == sat)).dropna()

sats = ('CH', 'GO', 'GR1', 'GR2', 'GF1', 'GF2', 'SWA', 'SWB', 'SWC')
fig, axes = plt.subplots(len(sats), 1, figsize=(10, 5), sharex=True)
empty = np.empty(df.index.shape)
empty[:] = np.nan
axes[0].plot(df.index, empty)
for sat, ax in zip(sats, axes):
    _df = alignments(df, sat=sat)
    for date in _df.index:
        ax.axvline(date)
    ax.set_yticks([])
    ax.set_ylabel(sat)
fig.subplots_adjust(hspace=0)
fig.suptitle("Plane alignments");